## Домашнее задание

Вам нужно написать следующий пайплайн:
1. Получить эмбединги пользователей с помощью SVD.
2. Подать их в нейронную сеть, которую вы делали на практическом занятии.
Она принимает на вход пользователей и эмбединг айтомов, далее объединяет их в два полносвязанных слоя. На последнем слое выдаёт прогноз с функцией активации сигмоид.
3. Обучить сеть.


In [ ]:
!pip install surprise

In [ ]:
### YOUR CODE HERE ###
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import KNNBaseline
from surprise.model_selection import train_test_split
data = Dataset.load_builtin('ml-100k')
from surprise import SVD



In [ ]:
import os
import tensorflow as tf
print(tf.__version__)

import sys
import keras
import keras.backend as K
import keras.layers as L
from keras.regularizers import l1, l2
import json
np.random.seed(42)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.3)

N_FACTORS = 8

algo = SVD(n_factors=N_FACTORS, biased=False, random_state=42)

predictions = algo.fit(trainset).test(testset)

df_all = pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
    try:
        #добавляем только те строки, где возможно преобразовать uid, iid
        uid = trainset.to_inner_uid(i.uid)
        iid = trainset.to_inner_iid(i.iid)
        df_all = df_all.append({'userID': uid, 'itemID': iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)
    except:
        pass

df_all2 = pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
    df_all2 = df_all2.append({'userID': i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

len(df_all), len(df_all2)


In [ ]:
df_all['userID'] = df_all['userID'].astype('int32')
df_all['itemID'] = df_all['itemID'].astype('int32')

df_all.head()

In [ ]:
df_all.describe()

In [ ]:
df_pivot = df_all.pivot(index='userID', columns='itemID', values='rating')
df_pivot

In [ ]:
len(list(trainset.all_items())), algo.qi.shape, len(list(trainset.all_users())), algo.pu.shape, len(df_all['userID'].unique()), len(df_all['itemID'].unique())

In [ ]:
algo.pu.shape, algo.qi.shape

In [ ]:
USERS_DIM = algo.pu.shape[0]
ITEMS_DIM = algo.qi.shape[0]
USERS_DIM, ITEMS_DIM

In [ ]:
import keras.layers as L

def build_model():
    U = L.Input(shape=(1,))
    I = L.Input(shape=(1,))
    user_latent = L.Flatten()(L.Embedding(USERS_DIM, N_FACTORS, weights=[algo.pu], trainable=False, mask_zero=True)(U))
    item_latent = L.Flatten()(L.Embedding(ITEMS_DIM, N_FACTORS, weights=[algo.qi], trainable=False, mask_zero=True)(I))
    vector = L.Multiply()([user_latent, item_latent])
    d = L.Dense(128, activation='relu')(vector)
    Y = L.Dense(1, activation='sigmoid')(d)
    return keras.models.Model(inputs=[U,I], outputs=Y)

model = build_model()
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='mse', metrics='mse')
model.summary()

In [ ]:
#Training
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_all, test_size=0.1)

Ut = df_train['userID'].values
It = df_train['itemID'].values

Yt = (df_train["real_rating"].values - 1.0)/4.0

Uv = df_test['userID'].values
Iv = df_test['itemID'].values

Yv = (df_test['real_rating'].values - 1.0)/4.0

len(Ut), len(Uv)

In [ ]:
Ut[:3], It[:3], Yt[:3]

In [ ]:
model = build_model()
model.compile(optimizer=keras.optimizers.Adam(lr=0.01), loss='binary_crossentropy', metrics='mse')
model.fit([Ut, It], Yt, batch_size=1, epochs=10, validation_data=([Uv, Iv], Yv))

In [ ]:
#Тестируем модель

testUserID = 0
testItemID = 0

pred = model.predict([np.array([testUserID]), np.array([testItemID])])
rating = pred*4 + 1
rating